In [2]:
from gensim.parsing.preprocessing import remove_stopwords
from  gensim.parsing.porter import PorterStemmer
from gensim import corpora
from gensim import models
from gensim import similarities
import xml.etree.ElementTree as ET
import os
from smart_open import open

In [1]:
#funcion que retorna una lista con los docs extraidos de los NAF
def xml_extract(carpeta):
    dicc_documents = {}
    for archivo in os.listdir(carpeta):
        direc = carpeta + '/' + archivo
        doc = ET.parse(direc)
        root = doc.getroot()
        Ids = root.find('*/public').get("publicId")
        raw = root.find('raw').text
        dicc_documents[Ids] = raw
    return dicc_documents



In [3]:
#funcion que normaliza, quita stopwords y hace steming a los doc
def process(text):
    p = PorterStemmer()
    doc_org = text.lower() #normalizo en minusculas
    doc_nor = remove_stopwords(doc_org)
    doc_stem = p.stem_sentence(doc_nor)
    return doc_stem.split()

In [4]:
def presicion(lista):
    cont = 1
    list_pk = []
    relev = 0
    for indice in lista:
        if indice == 1:
            relev+=1
        list_pk.append(relev /cont)
        cont+=1
    return list_pk

In [5]:
def recall(lista):
    k = 0
    list_rk = []
    for i in lista:
        if i == 1:
            k+=1
    relev = 0
    for indice in lista:
        if indice == 1:
            relev+=1
        list_rk.append(relev/k)
    return list_rk

In [6]:
def average(pres, recall):
    k = 0
    for i in recall:
        if i != 1:
            k+=1
    suma = 0
    for j in pres:
        if k > 0:
            suma = suma + j
        else:
            break
        k-=1
    return suma

In [7]:
def load_docs():
    dic_docs = {}#diccionario de documentos (id + contenido)
    dic_query = {}#diccionario de query (id + cont query)
    carpeta = 'C:/Users/Felipe Diaz/Documents/GitHub/Programas_python/Procesamiento de lenguaje/Motor_busqueda/prueba'#direccion de docs
    dic_docs = xml_extract(carpeta) #se llena dic, con los doc dentro de la carpeta
    carpeta = 'C:/Users/Felipe Diaz/Documents/GitHub/Programas_python/Procesamiento de lenguaje/Motor_busqueda/pquery'#direccion de queries
    dic_query = xml_extract(carpeta)#se llena dic, con los query dentro de la carpeta

    #se crea la lista con los documentos procesados
    docdict = [] #lista que va a contener el raw de los documentos
    for doc in dic_docs:#recorre diccionario de los documentos
        docdict.append(process(dic_docs[doc]))#se agrega el raw de los documentos procesados (normalizados)
    vocabulary_doc = corpora.Dictionary(docdict)#se crea un vocabulario del corpus a partir de la lista de docs
    vocabulary_doc.save('dic_documents.dict')#se guarda en un archivo .dict el vocabulario
    print("\nvocabulario:\n",vocabulary_doc)#se imprime las palabras del vocabulario
    print(vocabulary_doc.token2id)#se imprime token de las palabras (los asigna gensim)
    
    #seccion para crear el corpus como bolsa de palabras (se crean tuplas)
    class mycorpus(object):#clase de bolsa de palabras
        def __iter__(self):
            for doc in dic_docs:#se recorre diccionario de los documentos
                yield vocabulary_doc.doc2bow(process(dic_docs[doc]))#se crea bolsa de palabras
    
    corpus_memory_friendly = mycorpus()#se lama a la clase para crear instancia

    corpora.MmCorpus.serialize('corpus.mm',corpus_memory_friendly)#se almacena la bolsa de palabras en corpus.mm

    corpus = corpora.MmCorpus('corpus.mm')#se carga la bolsa de palabras (el corpus)
    print("\ncorpus\n",corpus)#se imprime el corpus

    #IDF Y TF
    vocabulary_doc = corpora.Dictionary.load('dic_documents.dict')#se carga el vocabulario
    corpus = corpora.MmCorpus('corpus.mm')#se carga el corpus (bolsa de palabras)
    tfidf = models.TfidfModel(corpus) #se crea modelo tf idf a partir del corpus (bolsa de palabras) se incializa

    #construimos la rep. vectorial de los query (bolsa de palabras)
    lis_doc_bow = []#lista donde se va a almacenar la BOW de los query
    for querii in dic_query:#se recorre dic de query para obtener el contenido
        query_doc_bow = vocabulary_doc.doc2bow(process(dic_query[querii]))#se crea bolsa de palabras a partir del query procesado
        query_doc_bow = tfidf[query_doc_bow]#modelo TFIDF de la BOW
        lis_doc_bow.append(query_doc_bow)#se agrega el modelo vect a la lista
    index = similarities.MatrixSimilarity(tfidf[corpus])# se construye matriz term/doc con los docs del corpus
    index.save('clase3tfidf.index')#se almacena en clase3tfidf.index 

    print("\nlista bolsa de palabras query:\n",lis_doc_bow)#se imprime BOW de los query

    index = similarities.MatrixSimilarity.load('clase3tfidf.index')#se carga la matriz term/doc

    simil_allquery = []#se crea lista para los tp 10 de similitudes
    for querii in lis_doc_bow:#se recorre lista de BOW de los queri
        sims = index[querii]#se almacena en sims la similitud entre el query evaluado por (querii) y la matriz term/doc (index)
        sims_ord = sorted(sims)#se ordena la lista obtenida de menor a mayor
        sims_ord.reverse()#se invierte la lista para obtener los mas similares
        simil_allquery.append(sims_ord[0:10])#se agrega la lista de los top 10 en otra lista
    print("\nsimilares", simil_allquery)#se imprime el resultado de los top 10 de todos los queries
    dic_final_top = {}
    cont = 0
    for key in dic_query:
        dic_final_top[key] = simil_allquery[cont]
        cont+=1
    print("dic final: ",dic_final_top)
    #crear archivo
    file = open("top10.txt","w")
    for elemento in dic_final_top:
        file.write('%s '%elemento)
        file.write('%s\n'%dic_final_top[elemento])
    file.close() 

    #crear diccionario de presicion
    dic_bin = {}
    for key in dic_final_top:#se recorre el diccionario final
        temp = dic_final_top[key]#se crea una variable temp que contiene la lista de valores similares al query
        print("temporal ",key," : ",temp)
        cont = 0
        for indice in temp: #se recorre variable temp (que es una lista) para evaluar si es igual a 0 o no y crear una lista binaria
            if indice != 0:
                temp[cont] = 1
            else:
                temp[cont] = 0
            cont+=1
            #print("lista_bin",lista_bin)
        dic_bin[key] = temp
    print("dic binario: ", dic_bin)

    #calcular la presicion para cada query
    dic_pres = {}
    for key in dic_bin:
        dic_pres[key] = presicion(dic_bin[key])
    print("dic presicion",dic_pres)

    #calcular el recall para cada query
    dic_rec = {}
    for key in dic_bin:
        dic_rec[key] = recall(dic_bin[key])
    print("dic recall: ", dic_rec)

    #average presicion
    dic_average = {}
    for key in dic_pres:
        dic_average[key] = average(dic_pres[key],dic_rec[key])
    print("average:",dic_average)



In [8]:
if __name__ == "__main__":
    load_docs()


0.49841744680530864), (272, 0.4367928111388928), (1030, 0.5295243567851534), (1525, 0.5295243567851534)]]

similares [[0.12143221, 0.07235707, 0.06686923, 0.056732856, 0.052190553, 0.047896527, 0.03725162, 0.033070035, 0.029168405, 0.028492954], [0.097560726, 0.069698766, 0.05081198, 0.044002477, 0.042062104, 0.03800237, 0.035594635, 0.027392177, 0.026838306, 0.026755294], [0.044162437, 0.038662996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.11964977, 0.094021976, 0.08856362, 0.0779949, 0.06499117, 0.054028586, 0.029282548, 0.020812385, 0.017697731, 0.017696753], [0.5088487, 0.35561892, 0.18223348, 0.157358, 0.09567668, 0.086457066, 0.07941789, 0.032588, 0.030296305, 0.026581798], [0.12826984, 0.12508929, 0.07105334, 0.056988798, 0.054285254, 0.042221483, 0.039243482, 0.038040724, 0.036858752, 0.036559604], [0.12593183, 0.124517955, 0.11882997, 0.10271681, 0.100186035, 0.075584576, 0.07050622, 0.05598318, 0.05273224, 0.05255995], [0.13511248, 0.0970497, 0.0965882, 0.08240115, 0.043301